# Pipeline for Preprocessing aorta data

In [7]:
from ipynb.fs.full.processing_Functions import Match_SurfacePoint_to_CenterlinePoint,Calculate_centerline_length,merge_df_SP_CLP_Match_cal
import pandas as pd
import numpy as np
import time
from ast import literal_eval
import matplotlib.pyplot as plt

## Workfolders and paths

In [3]:
work_Folder = **
Aorta_Metrics_file_name = 'aorta_clipped_metrics.dat'
Aorta_centerline_file_name = 'aorta_clgm.dat'

# full path
Metric_file_path = work_Folder + "\\" + Aorta_Metrics_file_name
Curve_file_path  = work_Folder + "\\" + Aorta_centerline_file_name

## Reading data in dataframe

In [4]:
df_SP = pd.read_csv(Metric_file_path,
                 sep="\s+", #separator whitespace
                 header=0) 

df_CLP = pd.read_csv(Curve_file_path,
                 sep="\s+", #separator whitespace
                 header=0) 


## Calculate the euclidian distance between surface and centerline

In [4]:
#long calculation! save to csv as backup
df_SP_CLP = Match_SurfacePoint_to_CenterlinePoint(df_SP[['X','Y','Z']],df_CLP[['X','Y','Z']],200)
df_SP_CLP.to_csv(work_Folder + "\\" + 'df_SP_CLP_match2.csv')

calculating (%): 0.0(0 out of 103766)
calculating (%): 9.6(10000 out of 103766)
calculating (%): 19.3(20000 out of 103766)
calculating (%): 28.9(30000 out of 103766)
calculating (%): 38.5(40000 out of 103766)
calculating (%): 48.2(50000 out of 103766)
calculating (%): 57.8(60000 out of 103766)
calculating (%): 67.5(70000 out of 103766)
calculating (%): 77.1(80000 out of 103766)
calculating (%): 86.7(90000 out of 103766)
calculating (%): 96.4(100000 out of 103766)


In [5]:
df_SP_CLP = pd.read_csv(work_Folder + "\\" + 'df_SP_CLP_match2.csv')

### centerline length

In [8]:
centerline_length = Calculate_centerline_length(df_CLP)
df_CLP['centerline_length'] = centerline_length

## Merge generated data for futher processing

In [9]:
#Merge all data in one large dataframe

df = merge_df_SP_CLP_Match_cal(df_SP,df_CLP,df_SP_CLP)

In [13]:
df.columns

Index(['X_SP', 'Y_SP', 'Z_SP', 'velocity_magnitude', 'x_velocity',
       'y_velocity', 'z_velocity', 'wall_shear', 'x_wall_shear',
       'y_wall_shear', 'z_wall_shear', 'pressure', 'velocity0', 'velocity1',
       'velocity2', 'GroupIds', 'AngularMetric', 'AbscissaMetric',
       'centerline_point_index', 'min_distance', 'point_vector', 'X_CLP',
       'Y_CLP', 'Z_CLP', 'MaximumInscribedSphereRadius', 'Abscissas',
       'ParallelTransportNormals0', 'ParallelTransportNormals1',
       'ParallelTransportNormals2', 'Curvature', 'Torsion', 'FrenetTangent0',
       'FrenetTangent1', 'FrenetTangent2', 'FrenetNormal0', 'FrenetNormal1',
       'FrenetNormal2', 'FrenetBinormal0', 'FrenetBinormal1',
       'FrenetBinormal2'],
      dtype='object')

In [10]:
area = Calculate_Surface_area(df)

In [11]:
df['bumpy'] = df['min_distance']-df['MaximumInscribedSphereRadius']

In [12]:
df_merged_area = df.merge(area,left_on = 'centerline_point_index', right_on = 'index_CL')
df_merged_area.to_csv(work_Folder + "\\" + 'Scan2_finalOutput.csv')

In [1]:
def Calculate_Surface_area(df_merged):
    #sort the dataframe first
    df_merged = df_merged.sort_values(['centerline_point_index','AngularMetric'],ascending=[True, False])
    
    # calculate the surface area for each centerlinepoint
    area_list = []
    Loop_list = df_merged['centerline_point_index'].unique()
    #Loop through all surface points which have a centerline assigned
    for i in Loop_list:
        #filter the dataframe for each centerline_point
        df_merged[df_merged['centerline_point_index']== i]
        Area = 0
        #get the amount of surface points assinged to a specifc centerline_point_index
        N_tri_edg = len(df_merged[df_merged['centerline_point_index']== i])
        for j in range(0,N_tri_edg):
            #check if j is last point, then cross last with first point
            if j == N_tri_edg:
                #vec_A = np.array(literal_eval(df_merged['point_vector'][j]))
                #vec_B = np.array(literal_eval(df_merged['point_vector'][0]))
                vec_A = np.array(literal_eval(df_merged['point_vector'][j]))
                vec_B = np.array(literal_eval(df_merged['point_vector'][0]))
                Area = Area + 0.5 * np.linalg.norm(np.cross(vec_A,vec_B))

            if j < N_tri_edg:
                #vec_A = np.array(literal_eval(df_merged['point_vector'][j]))
                #vec_B = np.array(literal_eval(df_merged['point_vector'][j+1]))
                vec_A = np.array(literal_eval(df_merged['point_vector'][j]))
                vec_B = np.array(literal_eval(df_merged['point_vector'][j+1]))
                Area = Area + 0.5 * np.linalg.norm(np.cross(vec_A,vec_B))
        
        area_list.append([Area,i])
    df_area = pd.DataFrame(area_list,  columns=['Area', 'index_CL'])   
    return(df_area)